https://github.com/modflowpy/flopy/blob/develop/examples/Tutorials/Tutorial02/tutorial02.py

In [94]:
import os
import numpy as np
import matplotlib.pyplot as plt
import flopy
%matplotlib inline
ws = 'C:/Users/Admin/Desktop/Chelsea/Projects/DelawareStream/FloPy/Model/Tutorial02'
if not os.path.exists(ws): #make sure workspace directory exists
    os.makedirs(ws)
name = 'Tutorial 02'
exe_name = os.path.abspath('C:/Users/Admin/Desktop/Chelsea/Projects/DelawareStream/Script/Executables/mf2005')

In [95]:
# Model domain and grid definition
Lx = 1000.
Ly = 1000.
ztop = 10.
zbot = -50.
nlay = 1
nrow = 10
ncol = 10
delr = Lx / ncol
delc = Ly / nrow
delv = (ztop - zbot) / nlay
botm = np.linspace(ztop, zbot, nlay + 1)
hk = 1.
vka = 1.
sy = 0.1
ss = 1.e-4
laytyp = 1

In [96]:
# Variables for the BAS package
# Note that changes from the previous tutorial!
ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)
strt = 10. * np.ones((nlay, nrow, ncol), dtype=np.float32)

In [97]:
# Time step parameters
nper = 3
perlen = [1, 100, 100]
nstp = [1, 100, 100]
steady = [True, False, False]

In [98]:
# Flopy objects
mf = flopy.modflow.Modflow(modelname=name, model_ws=ws, exe_name=exe_name)
dis = flopy.modflow.ModflowDis(mf, nlay, nrow, ncol, delr=delr, delc=delc,
                               top=ztop, botm=botm[1:],
                               nper=nper, perlen=perlen, nstp=nstp,
                               steady=steady)
bas = flopy.modflow.ModflowBas(mf, ibound=ibound, strt=strt)
lpf = flopy.modflow.ModflowLpf(mf, hk=hk, vka=vka, sy=sy, ss=ss, laytyp=laytyp,
                               ipakcb=53)
pcg = flopy.modflow.ModflowPcg(mf)

In [99]:
# Make list for stress period 1
stageleft = 10.
stageright = 10.
bound_sp1 = []
for il in range(nlay):
    condleft = hk * (stageleft - zbot) * delc
    condright = hk * (stageright - zbot) * delc
    for ir in range(nrow):
        bound_sp1.append([il, ir, 0, stageleft, condleft])
        bound_sp1.append([il, ir, ncol - 1, stageright, condright])
print('Adding ', len(bound_sp1), 'GHBs for stress period 1.')

Adding  20 GHBs for stress period 1.


In [100]:
# Make list for stress period 2
stageleft = 10.
stageright = 0.
condleft = hk * (stageleft - zbot) * delc
condright = hk * (stageright - zbot) * delc
bound_sp2 = []
for il in range(nlay):
    for ir in range(nrow):
        bound_sp2.append([il, ir, 0, stageleft, condleft])
        bound_sp2.append([il, ir, ncol - 1, stageright, condright])
print('Adding ', len(bound_sp2), 'GHBs for stress period 2.')

Adding  20 GHBs for stress period 2.


In [101]:
# We do not need to add a dictionary entry for stress period 3.
# Flopy will automatically take the list from stress period 2 and apply it
# to the end of the simulation
stress_period_data = {0: bound_sp1, 1: bound_sp2}

In [102]:
# Create the flopy ghb object
ghb = flopy.modflow.ModflowGhb(mf, stress_period_data=stress_period_data)

In [103]:
# Create the well package
# Remember to use zero-based layer, row, column indices!
pumping_rate = -500.
wel_sp1 = [[0, nrow/2 - 1, ncol/2 - 1, 0.]]
wel_sp2 = [[0, nrow/2 - 1, ncol/2 - 1, 0.]]
wel_sp3 = [[0, nrow/2 - 1, ncol/2 - 1, pumping_rate]]
stress_period_data = {0: wel_sp1, 1: wel_sp2, 2: wel_sp3}
wel = flopy.modflow.ModflowWel(mf, stress_period_data=stress_period_data)

In [104]:
# Output control
stress_period_data = {}
for kper in range(nper):
    for kstp in range(nstp[kper]):
        stress_period_data[(kper, kstp)] = ['save head',
                                            'save drawdown',
                                            'save budget',
                                            'print head',
                                            'print budget']
oc = flopy.modflow.ModflowOc(mf, stress_period_data=stress_period_data,
                             compact=True)


In [105]:
# Write the model input files
mf.write_input()

In [106]:
# Run the model
success, mfoutput = mf.run_model(silent=False, pause=False)
if not success:
    raise Exception('MODFLOW did not terminate normally.')

FloPy is using the following  executable to run the model: C:\Users\Admin\Desktop\Chelsea\Projects\DelawareStream\Script\Executables\mf2005.exe

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.12.00 2/3/2017                        

 Using NAME file: Tutorial 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2019/07/09 13:55:11


 *** ERROR OPENING FILE "Tutorial" ON UNIT    53
       SPECIFIED FILE STATUS: UNKNOWN
       SPECIFIED FILE FORMAT: UNFORMATTED         
       SPECIFIED FILE ACCESS: STREAM              
       SPECIFIED FILE ACTION: READWRITE           
  -- STOP EXECUTION (SGWF2BAS7OPEN)


Exception: MODFLOW did not terminate normally.